## Testing prometheus BERT models

In [ ]:
################################################################################
# Strings and constants
################################################################################

# Prometheus repo
PROMETHEUS_REPO = 'https://github.com/KCL-Health_NLP/prometheus.git'

# Machine specific local variables.
# Define these as a class, like the below.
# Select which local variables to use by assigning
# a class to LOCALS.

# Locals when on colab
class ColabLocals:
  PROMETHEUS_DIR = '/content/prometheus'
  TORCH_TARGET_DEVICE = 'cpu'

# Locals when on cusp_vm
class CuspVmLocals:
  PROMETHEUS_DIR = '/home/ZKraljevic/projects/angus/prometheus'
  TORCH_TARGET_DEVICE = 'cpu'

# Select which locals class to use
LOCALS = ColabLocals

# Dataset descriptions
# Define these as a class, like the below
# Select which dataset to use by assigning
# a class to DATA

# Imdb movie reviews dataset
# Standard sentiment classification task
# Contains around 5000 rows
# Note that on colab, you might need to limit data slice to 1000 or less
class ImdbDescription: 
  PATH = LOCALS.PROMETHEUS_DIR+'/sentence_classifier/sentence_classifier/imdb_5k_reviews.csv'
  TEXT_COL = 'review'
  LABEL_COL = 'sentiment'
  TEST_TEXTS = ['hello my name is link i am in love with princess zelda',
                'this is just a test sentence',
                'project']

  # We can limit the data we will use to just some of the dataset
  # Use None if you want to specify start and end of dataset 
  SLICE_FROM = None
  SLICE_TO = 100

  # We can limit the text that will be used from the text column
  # Use None if you want to specify start and end of the text
  TEXT_FROM = None
  TEXT_TO = 1000   

# Model related parameters
TOKENIZER = 'bert-base-uncased'
TRAINING_EPOCHS = 1
TRAINING_TEST_SIZE = 0.2

In [ ]:
################################################################################
# Imports
################################################################################

# Some initial imports needed for setting up
import sys
from os import path

In [ ]:
# Clone prometheus and add to the path
# if we don't already have it
if not path.exists(LOCALS.PROMETHEUS_DIR):
  !git clone $PROMETHEUS_REPO
if LOCALS.PROMETHEUS_DIR not in sys.path:
  sys.path.insert(0,LOCALS.PROMETHEUS_DIR)

In [ ]:
# Main imports

# these try blocks are sometimes probelmatic imports, not on some of machines
# being used

# transformers is needed by BERT.py
try:
  import transformers
except ImportError as e:
  !pip install transformers

# nltk is used by NLP_Utils.py, which is imported by BERT.py
try:
  import nltk
except ImportError as e:
  !pip install nltk

# spacy is used by NLP_Utils.py, which is imported by BERT.py
try:
  import spacy
except ImportError as e:
  !pip install spacy

# Prometheus classes
from sentence_classifier.sentence_classifier.BERT import train_BERT
from sentence_classifier.sentence_classifier.BERT import load_and_run_BERT

# Other imports
import pandas as pd
import torch

In [ ]:
################################################################################
# Load data and run Prometheus sentence classifier
################################################################################

# Select which dataset to use
DATA = ImdbDescription

# Read in some data for training
df = pd.read_csv(DATA.PATH,header=1)
df = df[DATA.SLICE_FROM:DATA.SLICE_TO]

df[DATA.TEXT_COL] = df[DATA.TEXT_COL].apply(
    lambda x: x.strip().lower()[DATA.TEXT_FROM:DATA.TEXT_TO]
    )
df.tail()

In [ ]:
# Train model
bert_model = train_BERT(sentences=df[DATA.TEXT_COL],
                        labels=df[DATA.LABEL_COL],
                        BERT_tokenizer=TOKENIZER,
                        test_size=TRAINING_TEST_SIZE,
                        n_epochs=TRAINING_EPOCHS,
                        output_dir=None)
bert_model['stats']

In [ ]:
# Run the model on some test sentences
# First need to move the model to the device being used
model=bert_model['model']
model.to(torch.device(LOCALS.TORCH_TARGET_DEVICE))

load_and_run_BERT(sentences=DATA.TEST_TEXTS,
                  trained_bert_model=model,
                  BERT_tokenizer=TOKENIZER)